In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Langkah 1: Definisikan Model Pydantic untuk Output Terstruktur

Model ini mendefinisikan **skema** yang akan dikembalikan oleh agen. Menggunakan `response_format` dengan Pydantic memastikan:
- ✅ Ekstraksi data yang aman tipe
- ✅ Validasi otomatis
- ✅ Tidak ada kesalahan parsing dari respons teks bebas
- ✅ Pengalihan kondisi yang mudah berdasarkan bidang


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Langkah 2: Buat Alat Pemesanan Hotel

Alat ini adalah apa yang akan dipanggil oleh **availability_agent** untuk memeriksa apakah kamar tersedia. Kami menggunakan dekorator `@ai_function` untuk:
- Mengubah fungsi Python menjadi alat yang dapat dipanggil oleh AI
- Secara otomatis menghasilkan skema JSON untuk LLM
- Menangani validasi parameter
- Mengaktifkan pemanggilan otomatis oleh agen

Untuk demo ini:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Memiliki kamar ✅
- **Semua kota lain** → Tidak ada kamar ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Langkah 3: Definisikan Fungsi Kondisi untuk Routing

Fungsi-fungsi ini memeriksa respons agen dan menentukan jalur mana yang akan diambil dalam alur kerja.

**Pola Utama:**
1. Periksa apakah pesan adalah `AgentExecutorResponse`
2. Parse output terstruktur (model Pydantic)
3. Kembalikan `True` atau `False` untuk mengontrol routing

Alur kerja akan mengevaluasi kondisi-kondisi ini pada **edges** untuk memutuskan executor mana yang akan dipanggil selanjutnya.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Langkah 4: Buat Executor Tampilan Kustom

Executor adalah komponen alur kerja yang melakukan transformasi atau efek samping. Kita menggunakan dekorator `@executor` untuk membuat executor kustom yang menampilkan hasil akhir.

**Konsep Utama:**
- `@executor(id="...")` - Mendaftarkan fungsi sebagai executor alur kerja
- `WorkflowContext[Never, str]` - Tipe petunjuk untuk input/output
- `ctx.yield_output(...)` - Menghasilkan hasil akhir alur kerja


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Langkah 5: Muat Variabel Lingkungan

Konfigurasikan klien LLM. Contoh ini bekerja dengan:
- **Model GitHub** (Tier Gratis dengan token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Langkah 6: Membuat Agen AI dengan Output Terstruktur

Kami membuat **tiga agen khusus**, masing-masing dibungkus dalam `AgentExecutor`:

1. **availability_agent** - Memeriksa ketersediaan hotel menggunakan alat
2. **alternative_agent** - Menyarankan kota alternatif (ketika tidak ada kamar)
3. **booking_agent** - Mendorong pemesanan (ketika kamar tersedia)

**Fitur Utama:**
- `tools=[hotel_booking]` - Memberikan alat kepada agen
- `response_format=PydanticModel` - Memaksa output JSON terstruktur
- `AgentExecutor(..., id="...")` - Membungkus agen untuk penggunaan alur kerja


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Langkah 7: Bangun Alur Kerja dengan Edge Kondisional

Sekarang kita menggunakan `WorkflowBuilder` untuk membangun grafik dengan pengalihan kondisional:

**Struktur Alur Kerja:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Metode Kunci:**
- `.set_start_executor(...)` - Menetapkan titik masuk
- `.add_edge(from, to, condition=...)` - Menambahkan edge kondisional
- `.build()` - Menyelesaikan alur kerja


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Langkah 8: Jalankan Kasus Uji 1 - Kota TANPA Ketersediaan (Paris)

Mari kita uji jalur **tanpa ketersediaan** dengan meminta hotel di Paris (yang tidak memiliki kamar dalam simulasi kami).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Langkah 9: Jalankan Kasus Uji 2 - Kota DENGAN Ketersediaan (Stockholm)

Sekarang mari kita uji jalur **ketersediaan** dengan meminta hotel di Stockholm (yang memiliki kamar dalam simulasi kami).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Intisari dan Langkah Berikutnya

### ✅ Apa yang Telah Anda Pelajari:

1. **Pola WorkflowBuilder**
   - Gunakan `.set_start_executor()` untuk mendefinisikan titik masuk
   - Gunakan `.add_edge(from, to, condition=...)` untuk routing kondisional
   - Panggil `.build()` untuk menyelesaikan workflow

2. **Routing Kondisional**
   - Fungsi kondisi memeriksa `AgentExecutorResponse`
   - Mengurai output terstruktur untuk membuat keputusan routing
   - Kembalikan `True` untuk mengaktifkan edge, `False` untuk melewatinya

3. **Integrasi Alat**
   - Gunakan `@ai_function` untuk mengubah fungsi Python menjadi alat AI
   - Agen memanggil alat secara otomatis saat dibutuhkan
   - Alat mengembalikan JSON yang dapat diurai oleh agen

4. **Output Terstruktur**
   - Gunakan model Pydantic untuk ekstraksi data yang tipe-aman
   - Setel `response_format=MyModel` saat membuat agen
   - Parselah respons dengan `Model.model_validate_json()`

5. **Eksekutor Kustom**
   - Gunakan `@executor(id="...")` untuk membuat komponen workflow
   - Eksekutor dapat mengubah data atau melakukan efek samping
   - Gunakan `ctx.yield_output()` untuk menghasilkan hasil workflow

### 🚀 Aplikasi Dunia Nyata:

- **Pemesan Perjalanan**: Periksa ketersediaan, sarankan alternatif, bandingkan opsi
- **Layanan Pelanggan**: Routing berdasarkan jenis masalah, sentimen, prioritas
- **E-commerce**: Cek inventaris, sarankan alternatif, proses pesanan
- **Moderasi Konten**: Routing berdasarkan skor toksisitas, flag pengguna
- **Workflow Persetujuan**: Routing berdasarkan jumlah, peran pengguna, tingkat risiko
- **Pemrosesan Multi-tahap**: Routing berdasarkan kualitas data, kelengkapan

### 📚 Langkah Berikutnya:

- Tambahkan kondisi yang lebih kompleks (multi kriteria)
- Implementasikan loop dengan manajemen status workflow
- Tambahkan sub-workflow untuk komponen yang dapat digunakan ulang
- Integrasikan dengan API nyata (pemesan hotel, sistem inventaris)
- Tambahkan penanganan kesalahan dan jalur fallback
- Visualisasikan workflow dengan alat visualisasi bawaan


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berupaya untuk mencapai akurasi, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidaktepatan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber otoritatif. Untuk informasi penting, disarankan menggunakan terjemahan profesional oleh penerjemah manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau salah tafsir yang mungkin timbul dari penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
